In [ ]:
import os
import glob
from skimage import io
import matplotlib.pyplot as plt

from skimage.segmentation import clear_border
from skimage.measure import regionprops
from skimage.morphology import label
import numpy as np

import imageio

In [ ]:
images_path = r'C:\Users\Pau\Desktop\Ground Truth AMR\export\image'
instance_path = r'C:\Users\Pau\Desktop\Ground Truth AMR\export\instance'
semantic_path = r'C:\Users\Pau\Desktop\Ground Truth AMR\export\semantic'

all_images =[]
for file in glob.glob(images_path + "\*.tif"):
    all_images.append(os.path.basename(file))

In [ ]:
def get_aim_props(fibre_labels, icml_class, axon_class):
    icml_list = []
    axon_list = []

    fibre_props = regionprops(fibre_labels)
    for i in range(len(fibre_props)):
        label_id = fibre_props[i].label # gets the label id of the current region
        instance_id_fibre = np.where(fibre_labels == label_id, label_id, 0)
        instance_id_icml = instance_id_fibre * icml_class
        instance_id_axon = instance_id_fibre * axon_class

        # append
        icml_list.append(instance_id_icml)
        axon_list.append(instance_id_axon)

    # reconstructs the label image, now containing the eroded labels
    icml_stack = np.stack(icml_list) # creates stack from list of images (numpy arrays)
    icml_labels = np.max(icml_stack, axis = 0) # calculates the maximum projection to get back a 2D, labelled image

    axon_stack = np.stack(axon_list) # creates stack from list of images (numpy arrays)
    axon_labels = np.max(axon_stack, axis = 0)

    # get region props
    icml_props = regionprops(icml_labels)
    axon_props = regionprops(axon_labels)

    #fig, ax = plt.subplots(1, 3, figsize=(18, 6))
    #ax[0].imshow(fibre_labels)
    #ax[1].imshow(icml_labels)
    #ax[2].imshow(axon_labels)
    #plt.show

    # get the are
    fibre_area = np.asarray([p['area'] for p in fibre_props])
    icml_area = np.asarray([p['area'] for p in icml_props])
    axon_area = np.asarray([p['area'] for p in axon_props]) 

    return fibre_area, icml_area, axon_area


In [ ]:

def g_ratio_and_it_fraction(fib_area, icml_area, axon_area):
    
    import math

    g_ratio = []
    it_fraction = []

    for i in range(len(fib_area)):
        # calculate g-ratio
        D = 2 * math.sqrt(fib_area[i] / math.pi)
        d = 2 * math.sqrt(icml_area[i] / math.pi)
        g_ratio.append(d/D)

        # calculate inner tongue fraction
        d_axon = 2 * math.sqrt(axon_area[i] / math.pi)
        it_fraction.append((d - d_axon) / (D - d_axon))
    
    return g_ratio, it_fraction


In [ ]:
def feature_mask(label_image, final_labels_image, feature_list, separators):
    class_list = []

    props = regionprops(label_image)
    for i in range(len(props)):
        label_id = props[i].label # gets the label id of the current region
        
        if(feature_list[i]<separators[0]):
            class_mask = np.where(label_image == label_id, 1, 0)
            final_class_mask = class_mask * final_labels_image
        elif(feature_list[i]<separators[1]):
            class_mask = np.where(label_image == label_id, 2, 0)
            final_class_mask = class_mask * final_labels_image
        elif(feature_list[i]<separators[2]):
            class_mask = np.where(label_image == label_id, 3, 0)
            final_class_mask = class_mask * final_labels_image
        else:
            class_mask = np.where(label_image == label_id, 4, 0)
            final_class_mask = class_mask * final_labels_image
        
        class_list.append(final_class_mask)

    # reconstructs the label image, now containing the eroded labels
    class_stack = np.stack(class_list) # creates stack from list of images (numpy arrays)
    feature_labels = np.max(class_stack, axis = 0) # calculates the maximum projection to get back a 2D, labelled image
    return feature_labels

def color_code_image(image, label_image):
    from skimage.color import label2rgb

    # define RGB colors from Tableau 10 (default in matplotlib)
    t10_blue = [31/255, 119/255, 180/255]
    t10_orange = [255/255, 127/255, 14/255]
    t10_green = [44/255, 160/255, 44/255]
    t10_red = [214/255, 39/255, 40/255]

    # list of colors
    colors=[t10_blue, t10_orange, t10_green, t10_red]

    # create RGB image with color-labels over the original, grayscale image
    rgb_labels = label2rgb(
        label=label_image,
        image=image,
        colors=colors,
        alpha=0.3,
        bg_label=0,
        bg_color=None
        )

    # plot original and color-coded images
    #fig, ax = plt.subplots(figsize=(16,16))
    #plt.imshow(rgb_labels)
    #plt.colorbar
    #plt.show()

    return rgb_labels

In [ ]:
g_ratio_pool = []
it_fraction_pool = []

for image_name in all_images:
    # import image
    image_filename = os.path.join(images_path, image_name)
    image = io.imread(image_filename)
    image = (image - np.min(image)) / (np.max(image) - np.min(image))
    
    # importa instance annotation image
    instance_filename = os.path.join(instance_path, image_name)
    instances = io.imread(instance_filename)
    
    # Transform to label matrix
    instances = label(instances)

    # clear objects connected to the border
    cleared_border = clear_border(instances)
    cleared_border = label(cleared_border)

    # import semantic annotation image
    semantic_filename = os.path.join(semantic_path, image_name)
    semantic = io.imread(semantic_filename)

    # get axon from semantic labels
    axon_pixels = np.where(semantic == 3, 1, 0)

    # get ICML from semantic labels
    icml_pixels = np.where(semantic >= 2, 1, 0)

    #fig, ax = plt.subplots(1, 3, figsize=(18, 6))
    #ax[0].imshow(cleared_border)
    #ax[1].imshow(icml_pixels)
    #ax[2].imshow(axon_pixels)
    #plt.show

    f_area, i_area, a_area = get_aim_props(cleared_border, icml_pixels, axon_pixels)
    g_ratio_now, it_fraction_now = g_ratio_and_it_fraction(f_area, i_area, a_area)

    # concatenate lists
    g_ratio_pool += g_ratio_now
    it_fraction_pool += it_fraction_now

    # plot image color-coded by g-ratio
    labels_g_ratio = feature_mask(cleared_border, axon_pixels, g_ratio_now, [0.8, 0.85, 0.9])
    color_code_g_ratio = color_code_image(image, labels_g_ratio)

    # plot image color-coded by it fraction
    labels_it_fraction = feature_mask(cleared_border, axon_pixels, it_fraction_now, [0.3, 0.5, 0.6])
    color_code_it_fraction = color_code_image(image, labels_it_fraction)

    imageio.imwrite(os.path.join(r'C:\Users\Pau\Desktop\Ground Truth AMR\export\color-labelled', image_name), color_code_it_fraction)

    fig, ax = plt.subplots(1, 2, figsize=(18, 9))
    fig.suptitle('Fibres grouped by myelin metrics\n' + image_name, fontsize=18)
    ax[0].set_title('g-ratio', fontsize=14)
    ax[0].imshow(color_code_g_ratio)
    ax[1].set_title('it fraction', fontsize=14)
    ax[1].imshow(color_code_it_fraction)
    plt.show


In [ ]:
assert len(it_fraction_pool) == len(g_ratio_pool)

plt.scatter(it_fraction_pool, g_ratio_pool)
plt.title("Myelin metrics")
plt.ylabel("g-ratio")
plt.xlabel("it fraction")
plt.show()